## 1. Check GPU and Install Dependencies

In [1]:
# Check GPU availability
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Install PyTorch with CUDA support (if not already installed)
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.9.0+cu126
CUDA available: False
CUDA version: 12.6


## 2. Clone and Install Glue Factory

In [3]:
# Clone the repository
!git clone https://github.com/cvg/glue-factory.git
%cd glue-factory

Cloning into 'glue-factory'...
remote: Enumerating objects: 2526, done.
remote: Counting objects: 100% (946/946), done.
remote: Compressing objects: 100% (220/220), done.
remote: Enumerating objects: 2526, done.
remote: Counting objects: 100% (946/946), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 2526 (delta 827), reused 726 (delta 726), pack-reused 1580 (from 2)
Receiving objects: 100% (2526/2526), 2.41 MiB | 9.63 MiB/s, done.
Resolving deltas: 100% (1808/1808), done.
/content/glue-factory
remote: Total 2526 (delta 827), reused 726 (delta 726), pack-reused 1580 (from 2)
Receiving objects: 100% (2526/2526), 2.41 MiB | 9.63 MiB/s, done.
Resolving deltas: 100% (1808/1808), done.
/content/glue-factory


In [5]:
# Install basic dependencies
!pip install -e .

Obtaining file:///content/glue-factory
  Installing build dependencies ... done
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/cvg/LightGlue.git to /tmp/pip-install-d4xmb23s/lightglue_026a3d76523e4efdb2cccd4923e9e60b
  Running command git clone --filter=blob:none --quiet https://github.com/cvg/LightGlue.git /tmp/pip-install-d4xmb23s/lightglue_026a3d76523e4efdb2cccd4923e9e60b
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/cvg/LightGlue.git to /tmp/pip-install-d4xmb23s/lightglue_026a3d76523e4efdb2cccd4923e9e60b
  Running command git clone --filter=blob:none --quiet https://github.com/cvg/LightGlue.git /tmp/pip-install-d4xmb23s/lightglue_026a3d76523e4efdb2

In [6]:
# Install extra dependencies for full functionality
!pip install -e .[extra]

Obtaining file:///content/glue-factory
  Installing build dependencies ... done
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/cvg/LightGlue.git to /tmp/pip-install-diwxe1_9/lightglue_81dd5b8f5add463680be19f9bb4973a7
  Running command git clone --filter=blob:none --quiet https://github.com/cvg/LightGlue.git /tmp/pip-install-diwxe1_9/lightglue_81dd5b8f5add463680be19f9bb4973a7
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/cvg/LightGlue.git to /tmp/pip-install-diwxe1_9/lightglue_81dd5b8f5add463680be19f9bb4973a7
  Running command git clone --filter=blob:none --quiet https://github.com/cvg/LightGlue.git /tmp/pip-install-diwxe1_9/lightglue_81dd5b8f5add463680

## 3. Mount Google Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

## 4. Setup HPatches Dataset from Google Drive

Assumes you have `hpatches-sequences-release` folder in your Google Drive.

In [9]:
import os
from pathlib import Path

# Set paths - MODIFY THIS to match your Google Drive structure
GDRIVE_HPATCHES = '/content/drive/MyDrive/hpatches-sequences-release'
LOCAL_DATA_DIR = '/content/glue-factory/data'

# Create data directory
!mkdir -p {LOCAL_DATA_DIR}

# Create symbolic link to HPatches in Google Drive
if os.path.exists(GDRIVE_HPATCHES):
    !ln -s {GDRIVE_HPATCHES} {LOCAL_DATA_DIR}/hpatches-sequences-release
    print(f"✅ Linked HPatches from Google Drive")
    print(f"   Source: {GDRIVE_HPATCHES}")
    print(f"   Target: {LOCAL_DATA_DIR}/hpatches-sequences-release")
else:
    print(f"❌ HPatches not found at {GDRIVE_HPATCHES}")
    print(f"   Please upload hpatches-sequences-release to your Google Drive")
    print(f"   Or modify GDRIVE_HPATCHES variable to point to correct location")

❌ HPatches not found at /content/drive/MyDrive/hpatches-sequences-release
   Please upload hpatches-sequences-release to your Google Drive
   Or modify GDRIVE_HPATCHES variable to point to correct location


In [ ]:
# Verify HPatches dataset structure
!ls -la {LOCAL_DATA_DIR}/hpatches-sequences-release/ | head -20

## 6. Training Setup

**Important**: Training requires large datasets:
- Homography pre-training: Oxford-Paris 1M images (~450 GB)
- Fine-tuning: MegaDepth (~420 GB)

HPatches is NOT used for training, only evaluation.

### 6.1 Setup Training Datasets in Google Drive

If you have training datasets in Google Drive, set up paths:

In [ ]:
# Extract all images from HPatches sequences into a single folder for training
import os
from pathlib import Path
import shutil

# Create training image directory
TRAIN_IMAGES_DIR = '/content/glue-factory/data/hpatches_images'
!mkdir -p {TRAIN_IMAGES_DIR}

# Copy all images from HPatches sequences with unique names
hpatches_dir = f"{LOCAL_DATA_DIR}/hpatches-sequences-release"
sequences = [d for d in os.listdir(hpatches_dir) if os.path.isdir(os.path.join(hpatches_dir, d))]

print(f"Found {len(sequences)} sequences in HPatches")
total_images = 0

for seq in sequences:
    seq_path = os.path.join(hpatches_dir, seq)
    # Copy all .ppm images with renamed filenames to avoid overwriting
    for img_file in Path(seq_path).glob("*.ppm"):
        # Create unique filename: sequence_name + original_filename
        # Example: v_abstract_1.ppm instead of just 1.ppm
        new_filename = f"{seq}_{img_file.name}"
        destination = os.path.join(TRAIN_IMAGES_DIR, new_filename)
        shutil.copy(img_file, destination)
        total_images += 1

print(f"✅ Copied {total_images} images to {TRAIN_IMAGES_DIR}")
print(f"   All images renamed with sequence prefix to avoid overwrites")

### 5.2 Create Training Configuration File

In [ ]:
# Create image list file required by the training script
import os

# Generate list of all images in training directory
image_list_path = f"{TRAIN_IMAGES_DIR}/revisitop1m.txt"
image_files = [f for f in os.listdir(TRAIN_IMAGES_DIR) if f.endswith('.ppm')]

# Write image list to file (just filenames, one per line)
with open(image_list_path, 'w') as f:
    for img in sorted(image_files):
        f.write(f"{img}\n")

print(f"✅ Created image list with {len(image_files)} images")
print(f"   Saved to: {image_list_path}")
print(f"   First 5 images: {image_files[:5]}")

In [ ]:
# Create custom config for training on HPatches images
config_content = """
data:
    name: homographies
    data_dir: hpatches_images
    image_dir: ""  # Images are in root directory, not in jpg/ subdirectory
    train_size: 5000
    val_size: 0  # No validation split needed for this training
    batch_size: 16  # Reduced for Colab GPU
    num_workers: 2
    homography:
        difficulty: 0.7
        max_angle: 45
    photometric:
        name: lg
model:
    name: two_view_pipeline
    extractor:
        name: extractors.superpoint_open
        max_num_keypoints: 512
        force_num_keypoints: True
        detection_threshold: 0.0
        nms_radius: 3
        trainable: False
    ground_truth:
        name: matchers.homography_matcher
        th_positive: 3
        th_negative: 3
    matcher:
        name: matchers.lightglue
        filter_threshold: 0.1
        flash: false
        checkpointed: true
train:
    seed: 0
    epochs: 20  # Reduced to 20 epochs as requested
    log_every_iter: 50
    eval_every_iter: 1000  # Disabled frequent validation
    save_every_iter: 2000
    test_every_epoch: 0  # Disable validation checks
    lr: 1e-4
    lr_schedule:
        start: 10
        type: exp
        on_epoch: true
        exp_div_10: 10
    plot: [5, 'gluefactory.visualization.visualize_batch.make_match_figures']
benchmarks:
    hpatches:
      eval:
        estimator: opencv
        ransac_th: 0.5
"""

# Save config file
config_path = "/content/glue-factory/gluefactory/configs/hpatches_training.yaml"
with open(config_path, 'w') as f:
    f.write(config_content)

print(f"✅ Created training config at {config_path}")
print(f"   - Epochs: 20")
print(f"   - Validation: Disabled (no validation split)")
print(f"   - Checkpoints saved every 2000 iterations")

### 5.3 Start Training

This will train LightGlue on your HPatches images using synthetic homographies.

In [ ]:
# Train the model
!python -m gluefactory.train hpatches_lightglue \
    --conf gluefactory/configs/hpatches_training.yaml

## 6. Evaluate the Trained Model

In [ ]:
# Install PoseLib for better evaluation
!pip install poselib

In [ ]:
# Evaluate your trained model on HPatches
!python -m gluefactory.eval.hpatches \
    --checkpoint hpatches_lightglue \
    --overwrite \
    eval.estimator=poselib \
    eval.ransac_th=-1

## 7. Save Trained Model to Google Drive

In [ ]:
# Save training outputs and model to Google Drive
OUTPUT_DIR = '/content/drive/MyDrive/glue-factory-trained-model'
!mkdir -p {OUTPUT_DIR}

# Copy the entire experiment folder
!cp -r outputs/training/hpatches_lightglue {OUTPUT_DIR}/
print(f"✅ Model saved to {OUTPUT_DIR}/hpatches_lightglue")

# Also save evaluation results
!cp -r outputs/results {OUTPUT_DIR}/results
print(f"✅ Evaluation results saved to {OUTPUT_DIR}/results")

## 8. (Optional) Visualize Results

In [ ]:
# View training logs
!cat outputs/training/hpatches_lightglue/training.log | tail -50

## 9. Monitor Training with TensorBoard (Optional)

In [ ]:
# Load TensorBoard to monitor training
%load_ext tensorboard
%tensorboard --logdir outputs/training/hpatches_lightglue

## Summary

This notebook trains a LightGlue model directly on your HPatches images:

**What it does:**
1. ✅ Mounts Google Drive and accesses your HPatches dataset
2. ✅ Extracts all images from HPatches sequences
3. ✅ Trains LightGlue using synthetic homographies on your images
4. ✅ Evaluates the trained model
5. ✅ Saves the trained model to Google Drive

**Training Details:**
- **Dataset:** Your HPatches images (~5000 image pairs)
- **Method:** Synthetic homographies for data augmentation
- **Model:** LightGlue with SuperPoint features
- **Epochs:** 30 (adjustable)
- **Batch Size:** 16 (for free Colab GPU)
- **Time:** ~2-3 hours on Colab T4 GPU

**Output:**
- Trained model weights in Google Drive
- Evaluation metrics (homography error, matching accuracy)
- Training logs and visualizations

**Next Steps:**
- Use the trained model for your own image matching tasks
- Adjust hyperparameters for better performance
- Evaluate on other datasets

**References:**
- [Glue Factory GitHub](https://github.com/cvg/glue-factory)
- [LightGlue Paper](https://arxiv.org/abs/2306.13643)

## 10. Test Your Trained Model on Custom Image Pairs

Use your trained model to match features between any two images.

In [ ]:
# Load your trained model for inference from Google Drive
import torch
from pathlib import Path
from gluefactory.models import get_model
from gluefactory.settings import DATA_PATH
from omegaconf import OmegaConf
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Mount Google Drive if not already mounted
from google.colab import drive
drive.mount('/content/drive')

# Path to your saved model in Google Drive
checkpoint_dir = Path("/content/drive/MyDrive/glue-factory-trained-model/hpatches_lightglue")
checkpoint_path = checkpoint_dir / "checkpoint_best.tar"

# Load config
config = OmegaConf.load(checkpoint_dir / "config.yaml")

# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = get_model(config.model.name)(config.model).eval().to(device)

# Load trained weights
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model'], strict=False)

print(f"✅ Loaded trained model from Google Drive")
print(f"   Path: {checkpoint_path}")
print(f"   Device: {device}")
print(f"   Epoch: {checkpoint['epoch']}")

In [ ]:
# Helper function to preprocess images
def load_and_preprocess_image(image_path, resize=480):
    """Load and preprocess an image for the model."""
    # Read image
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    # Convert BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize (short side to target size)
    h, w = img.shape[:2]
    scale = resize / min(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    img = cv2.resize(img, (new_w, new_h))
    
    # Convert to grayscale for feature extraction
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Normalize to [0, 1] and convert to tensor
    gray_tensor = torch.from_numpy(gray).float() / 255.0
    gray_tensor = gray_tensor.unsqueeze(0).unsqueeze(0).to(device)  # [1, 1, H, W]
    
    return {
        'image': gray_tensor,
        'image_rgb': img,
        'image_size': torch.tensor([[new_h, new_w]]).to(device)
    }

print("✅ Image preprocessing function ready")

In [ ]:
# Option 1: Test on images from HPatches in Google Drive
# Make sure HPatches is in your Google Drive
GDRIVE_HPATCHES = '/content/drive/MyDrive/hpatches-sequences-release'

# Pick a sequence (e.g., v_abstract)
test_sequence = "v_abstract"
image1_path = f"{GDRIVE_HPATCHES}/{test_sequence}/1.ppm"
image2_path = f"{GDRIVE_HPATCHES}/{test_sequence}/2.ppm"

# OR Option 2: Upload your own images (uncomment below)
# from google.colab import files
# print("Upload first image:")
# uploaded1 = files.upload()
# image1_path = list(uploaded1.keys())[0]
# print("Upload second image:")
# uploaded2 = files.upload()
# image2_path = list(uploaded2.keys())[0]

print(f"Testing on:\n  Image 1: {image1_path}\n  Image 2: {image2_path}")

In [ ]:
# Run inference on the image pair
with torch.no_grad():
    # Load and preprocess images
    data0 = load_and_preprocess_image(image1_path)
    data1 = load_and_preprocess_image(image2_path)
    
    # Prepare batch for model
    batch = {
        'view0': {'image': data0['image']},
        'view1': {'image': data1['image']}
    }
    
    # Run model
    pred = model(batch)
    
    # Extract results
    kpts0 = pred['keypoints0'][0].cpu().numpy()  # [N, 2]
    kpts1 = pred['keypoints1'][0].cpu().numpy()  # [M, 2]
    matches = pred['matches0'][0].cpu().numpy()  # [N]
    confidence = pred['matching_scores0'][0].cpu().numpy()  # [N]
    
    # Get valid matches (matches > -1)
    valid = matches > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches[valid]]
    mconf = confidence[valid]

print(f"\n✅ Matching complete!")
print(f"   Keypoints in image 1: {len(kpts0)}")
print(f"   Keypoints in image 2: {len(kpts1)}")
print(f"   Matches found: {len(mkpts0)}")
print(f"   Average confidence: {mconf.mean():.3f}")

In [ ]:
# Visualize the matches
def plot_matches(img0, img1, kpts0, kpts1, color=None, kp_size=2, line_width=1):
    """Plot matched keypoints between two images."""
    h0, w0 = img0.shape[:2]
    h1, w1 = img1.shape[:2]
    
    # Create side-by-side image
    h_max = max(h0, h1)
    img_combined = np.zeros((h_max, w0 + w1, 3), dtype=np.uint8)
    img_combined[:h0, :w0] = img0
    img_combined[:h1, w0:] = img1
    
    # Offset keypoints in second image
    kpts1_offset = kpts1.copy()
    kpts1_offset[:, 0] += w0
    
    # Plot matches
    plt.figure(figsize=(20, 10))
    plt.imshow(img_combined)
    
    # Draw keypoints
    plt.scatter(kpts0[:, 0], kpts0[:, 1], c='red', s=kp_size, alpha=0.5)
    plt.scatter(kpts1_offset[:, 0], kpts1_offset[:, 1], c='red', s=kp_size, alpha=0.5)
    
    # Draw match lines
    if color is None:
        color = 'lime'
    
    for i in range(len(kpts0)):
        if i < 200:  # Limit to first 200 matches for clarity
            plt.plot([kpts0[i, 0], kpts1_offset[i, 0]], 
                    [kpts0[i, 1], kpts1_offset[i, 1]], 
                    color=color, linewidth=line_width, alpha=0.4)
    
    plt.axis('off')
    plt.title(f'Feature Matches: {len(kpts0)} matches (showing first 200)', fontsize=16)
    plt.tight_layout()
    plt.show()

# Plot the results
plot_matches(data0['image_rgb'], data1['image_rgb'], mkpts0, mkpts1)

### Upload and Test Your Own Images

You can also upload your own image pairs from your computer:

In [ ]:
# Upload your own images
from google.colab import files
import os

print("Upload first image:")
uploaded1 = files.upload()
img1_name = list(uploaded1.keys())[0]
img1_path = os.path.join('/content', img1_name)

print("\nUpload second image:")
uploaded2 = files.upload()
img2_name = list(uploaded2.keys())[0]

img2_path = os.path.join('/content', img2_name)

print(f"   Image 2: {img2_path} (exists: {os.path.exists(img2_path)})")
print(f"   Image 1: {img1_path} (exists: {os.path.exists(img1_path)})")
print(f"\n✅ Images uploaded:")

In [ ]:
# Match your uploaded images
with torch.no_grad():
    data0 = load_and_preprocess_image(img1_path)
    data1 = load_and_preprocess_image(img2_path)
    
    batch = {
        'view0': {'image': data0['image']},
        'view1': {'image': data1['image']}
    }
    
    pred = model(batch)
    
    kpts0 = pred['keypoints0'][0].cpu().numpy()
    kpts1 = pred['keypoints1'][0].cpu().numpy()
    matches = pred['matches0'][0].cpu().numpy()
    confidence = pred['matching_scores0'][0].cpu().numpy()
    
    valid = matches > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches[valid]]
    mconf = confidence[valid]

print(f"Matches found: {len(mkpts0)} (avg confidence: {mconf.mean():.3f})")
plot_matches(data0['image_rgb'], data1['image_rgb'], mkpts0, mkpts1)